In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import sys
from tqdm.notebook import tqdm
import math
from IPython.display import FileLink
import gzip
import os
import random


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#log_f = open('running_log','w+')


"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break"""


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

for dirname, _, filenames in os.walk('/kaggle/input/ecg-retrieval-static-integrated'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
try:
    for line in open('/kaggle/input/ecg-retrieval-static-integrated/running_log'):
        print(line.strip())
except:
    pass

count_label_dict = {82: 51162, 81: 14897, 57: 14733, 83: 11989, 46: 8344, 67: 6867, 90: 6573, 8: 6269, 61: 5412, 49: 5202, 76: 4484, 78: 4030, 80: 3649, 2: 3389, 84: 2996, 0: 2847, 75: 2826, 66: 2724, 58: 2597, 24: 2249, 51: 2104, 33: 1972, 42: 1947, 114: 1932, 86: 1886, 68: 1822, 110: 1718, 69: 1665, 62: 1409, 115: 1380, 91: 1361, 71: 1354, 17: 1324, 44: 1290, 47: 1255, 85: 1161, 60: 1105, 41: 1043, 87: 980, 94: 921, 10: 802, 88: 738, 19: 682, 35: 668, 116: 644, 118: 644, 98: 558, 121: 552, 7: 428, 100: 413, 106: 410, 63: 392, 95: 367, 70: 353, 6: 324, 103: 315, 104: 310, 64: 300, 15: 278, 117: 276, 55: 276, 122: 269, 77: 260, 107: 251, 14: 247, 32: 220, 48: 202, 27: 199, 13: 189, 119: 184, 74: 177, 21: 161, 34: 156, 52: 110, 26: 106, 96: 103, 113: 99, 16: 94, 120: 92, 79: 83, 109: 81, 108: 74, 43: 72, 11: 69, 1: 68, 23: 67, 9: 60, 12: 52, 45: 47, 72: 46, 112: 46, 101: 44, 38: 32, 102: 28, 93: 23, 4: 19, 111: 14, 59: 14, 73: 14, 29: 13, 89: 12, 65: 10, 105: 7, 40: 6, 97: 5, 31: 5, 39: 4, 5: 2, 99: 2, 25: 2, 50: 1, 36: 1, 20: 1}

In [2]:
sum[1 for line in open('/kaggle/input/ecg-analysis/ideal_dict')]

SyntaxError: invalid syntax (<ipython-input-2-f90ac1a6cac2>, line 1)

In [3]:
"""def check_line(f_name):
        
        f1 = '../input/prepare-integrated-data1/' + f_name
        f2 = '../input/static-integrated-data1/' + f_name
        line1 = open(f1).read()
        line2 = open(f2).read()
        try:
            file_name1, index_data1, index_label1, index_feature1 = line1.strip().split('\t')
            file_name2, index_data2, index_label2, index_feature2, features2 = line2.strip().split('\t')
        except: 
            return
        
        assert file_name1 == file_name2
        assert index_data1 == index_data2
        assert index_label1 == index_label2
        assert index_feature1 == index_feature2
        
for filename in tqdm(os.listdir('/kaggle/input/prepare-integrated-data1')):
    check_line(filename)"""

"def check_line(f_name):\n        \n        f1 = '../input/prepare-integrated-data1/' + f_name\n        f2 = '../input/static-integrated-data1/' + f_name\n        line1 = open(f1).read()\n        line2 = open(f2).read()\n        try:\n            file_name1, index_data1, index_label1, index_feature1 = line1.strip().split('\t')\n            file_name2, index_data2, index_label2, index_feature2, features2 = line2.strip().split('\t')\n        except: \n            return\n        \n        assert file_name1 == file_name2\n        assert index_data1 == index_data2\n        assert index_label1 == index_label2\n        assert index_feature1 == index_feature2\n        \nfor filename in tqdm(os.listdir('/kaggle/input/prepare-integrated-data1')):\n    check_line(filename)"

In [4]:
"""data schema:
key, base64_data, base64_label, features
"""
import multiprocessing

path = {}

download_and_use_pretrained_model = False

is_load_from_pretrain = False
path['pretrain_model_path'] = '/kaggle/input/kernel8dd5b9b242/step7224'

shuffle = True
is_gzip_file = False
valid_ratio = 0.05  #5948 samples
test_ratio = 0.05   #5948 samples
train_ratio = 1 - valid_ratio - test_ratio   #107064 samples
worker_num = multiprocessing.cpu_count()
#worker_num = 8

topk = 5
feature_index = 0
label_index = 1

epochs = 40
batch_size = 128
learning_rate = 1e-3
is_warmup_decay = 1
warmup_ratio = 0.1
threshold = 0.5
sigmoid_input = False

use_selected_label = False
select_label = [80,81,82,8,83]

path['use_resnet'] = 'resnet34'

use_l2_penalty = False
adam_epsilon = 1e-8

use_float32 = False

leads = 12
num_samples = 118960
num_class = 123
    
if use_selected_label: 
    num_class = len(select_label)

steps_per_epochs = int(num_samples/batch_size)
num_steps = steps_per_epochs * epochs
save_step = int(num_steps/10)
decay_step = [int(num_steps*0.6), int(num_steps*0.8)]
warmup_step = max(2*steps_per_epochs, int(num_steps/10))
#evaluate_step = int(num_steps/10)
evaluate_step = steps_per_epochs

In [5]:
#load train file list, valid file list and test file list

path = {}
path['input_data'] = ['/kaggle/input/static-integrated-data1',
                      '/kaggle/input/static-integrated-data2',
                      '/kaggle/input/static-integrated-data3',
                      '/kaggle/input/static-integrated-data4']

def get_train_val_test_list(paths):
    files = []
    for dir_add in paths:
        f_adds = os.listdir(dir_add)
        f_adds = sorted(f_adds)
        for i,f_add in enumerate(f_adds.copy()):
            if '.part' not in f_add:
                f_adds.remove(f_add)
            else:
                f_add = dir_add + '/' + f_add
                files.append(f_add)
    num_samples = len(files)
    num_valids = int(valid_ratio * num_samples)
    num_tests = int(test_ratio * num_samples)
    num_trains = num_samples - num_valids - num_tests
    #return train_list, valid_list, test_list
    return files[:num_trains], files[num_trains:num_trains+num_valids], files[num_trains+num_valids:]
    
path['train_list'], path['valid_list'], path['test_list'] = get_train_val_test_list(path['input_data'])

In [6]:
print(len(path['train_list']))
print(len(path['valid_list']))
print(len(path['test_list']))

107064
5948
5948


In [7]:
import base64

def parse_line(f_add):
    
    with open(f_add, 'r') as f:
        line = f.read()
        try:
            index_name, index_data, index_label, index_feature, features = line.strip().split('\t')
        except:
            index_name, index_data, index_label, index_feature = line.strip().split('\t')
            features = None
        
        index_data = base64.b64decode(index_data)
        index_data = np.frombuffer(index_data, dtype=np.float16).reshape(-1,12).transpose()
        
        index_label = base64.b64decode(index_label)
        index_label = np.frombuffer(index_label, dtype=np.float16)
        
        if features != None:
            features = base64.b64decode(features)
            #features = np.frombuffer(features, dtype=np.float16)
            features = np.frombuffer(features, dtype=np.float16)
        else:
            features = np.array([0] * feature_size)
            
        index_data = index_data.astype(np.float64)
        index_label = index_label.astype(np.float64)
        
        features = features.astype(np.float64)
        features[~np.isfinite(features)] = 0
        
        if use_selected_label:
            index_label = np.asarray([index_label[i] for i in select_label], dtype=np.float16)
                    
        assert len(index_label) == num_class
        
        sample = {}
        sample['file_name'] = f_add
        sample['index_name'] = index_name
        sample['index_data'] = index_data
        sample['index_label'] = index_label
        sample['index_feature'] = index_feature
        sample['features'] = features
        
        return sample

In [8]:
"""train_datas = []
for train_name in tqdm(random.sample(path['train_list'], 30000)):
    train_sample = parse_line(train_name)
    train_datas.append(train_sample["index_data"])
    
def normCorrelation(data, sample2):
    corr = []
    for a, b in zip(data, sample2["index_data"]):
        #corr.append(abs(np.corrcoef(a.ravel(), b.ravel())[0][1]))
        a = (a - np.mean(a)) / (np.std(a) * len(a))
        b = (b - np.mean(b)) / (np.std(b))
        v = np.correlate(a, b, 'valid')
        v = 0 if np.isfinite(v) == False else v
        assert np.isfinite(v) == True
        corr.append(abs(v))
        break
    return sum(corr) / len(corr)

def getIdealCoe(test_file):
    test_sample = parse_line(test_file)
    test_name = test_file[test_file.rindex('/')+1:]
    corrs = []
    #for train_file in tqdm(random.sample(path['train_list'], 1000)):
    for train_data in train_datas:
        corr = normCorrelation(train_data, test_sample)
        corrs.append(corr)
    corrs.sort(reverse=True)
    out_line = [test_name,str(corrs[0])[1:-1],str(corrs[1])[1:-1],str(corrs[2])[1:-1],str(corrs[3])[1:-1],str(corrs[4])[1:-1]]
    return '\t'.join(out_line) + os.linesep


with open('ideal_dict',"w+") as f:
    for filename in tqdm(path['test_list'][:3000]):
        line = getIdealCoe(filename)
        f.write(line)"""

'train_datas = []\nfor train_name in tqdm(random.sample(path[\'train_list\'], 30000)):\n    train_sample = parse_line(train_name)\n    train_datas.append(train_sample["index_data"])\n    \ndef normCorrelation(data, sample2):\n    corr = []\n    for a, b in zip(data, sample2["index_data"]):\n        #corr.append(abs(np.corrcoef(a.ravel(), b.ravel())[0][1]))\n        a = (a - np.mean(a)) / (np.std(a) * len(a))\n        b = (b - np.mean(b)) / (np.std(b))\n        v = np.correlate(a, b, \'valid\')\n        v = 0 if np.isfinite(v) == False else v\n        assert np.isfinite(v) == True\n        corr.append(abs(v))\n        break\n    return sum(corr) / len(corr)\n\ndef getIdealCoe(test_file):\n    test_sample = parse_line(test_file)\n    test_name = test_file[test_file.rindex(\'/\')+1:]\n    corrs = []\n    #for train_file in tqdm(random.sample(path[\'train_list\'], 1000)):\n    for train_data in train_datas:\n        corr = normCorrelation(train_data, test_sample)\n        corrs.append(corr

In [9]:
train_labels = []
for train_name in tqdm(path['train_list']):
    train_sample = parse_line(train_name)
    train_labels.append(train_sample["index_label"])

In [10]:
from sklearn.metrics import f1_score
#计算F1score
def calc_f1(y_true, y_pre, threshold=0.5):
    y_true = np.asarray(y_true).astype(np.int)
    y_pre = np.asarray(y_pre) > threshold
    return f1_score(y_true, y_pre)

def f1_loss(y_true, y_pred, beta=1) -> np.float32:
        
    tp = (y_true * y_pred).sum()
    tn = ((1 - y_true) * (1 - y_pred)).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = (1 + beta**2)* (precision*recall) / (beta**2 * precision + recall + epsilon)

    return round(f1, 2)

def getIdealF1(test_file):
    test_sample = parse_line(test_file)
    test_name = test_file[test_file.rindex('/')+1:]
    f1s = []
    """for train_file in tqdm(path['train_list']):
        train_sample = parse_line(train_file)
        f1 = calc_f1(train_sample["index_label"], test_sample["index_label"])
        f1s.append(f1)"""
    for train_label in train_labels:
        #f1 = calc_f1(train_label, test_sample["index_label"])
        f1s.append(f1_loss(train_label, test_sample["index_label"]))
    f1s.sort(reverse=True)
    #out_line = [test_name,str(corrs[0])[1:-1],str(corrs[1])[1:-1],str(corrs[2])[1:-1],str(corrs[3])[1:-1],str(corrs[4])[1:-1]]
    out_line = [test_name] + [str(f1s[i]) for i in range(10)]
    return '\t'.join(out_line) + os.linesep

with open('ideal_dict_f1_123label',"w+") as f:
    for filename in tqdm(path['test_list']):
        line = getIdealF1(filename)
        f.write(line)